Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Read data

In [ ]:
ad = AllData()
ad.usefanom = True
ad.gather(update=False)
#Select subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(0,2),esm=slice(0,2))

fe = FullEnsemble(ad)
fe.gather(force_update=True)

#try:
#    fe = fe.drop_sel(ism='PISM_DMI')
#except:
#    pass

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.15,bottom=.2)

alpha = .3 #Transparency of uncertainty range
perc = [17,83] #Percentiles of uncertainty range

Make and save figure

In [ ]:
fig,ax = plt.subplots(1,3,sharex=True,sharey='row',figsize=(7,3))

for s,ss in enumerate(ad.ssp):
    #Plot no feedback
    ax[s].plot(fe.time,100*np.median(fe.slr_nf[s,:,:,:],axis=(0,1)),c='.5',lw=2,label='No feedback')
    ax[s].fill_between(fe.time,100*np.percentile(fe.slr_nf[s,:,:,:],perc[0],axis=(0,1)),100*np.percentile(fe.slr_nf[s,:,:,:],perc[1],axis=(0,1)),color='.5',alpha=alpha,label=perc)

    #Plot with feedback
    ax[s].plot(fe.time,100*np.median(fe.slr_wf[s,:,:,:],axis=(0,1)),c=ad.scol[ss],lw=2,label='With feedback')
    ax[s].fill_between(fe.time,100*np.percentile(fe.slr_wf[s,:,:,:],perc[0],axis=(0,1)),100*np.percentile(fe.slr_wf[s,:,:,:],perc[1],axis=(0,1)),color=ad.scol[ss],alpha=alpha)

    ax[s].set_title(f'SSP-{ss}')
    print(100*np.median(fe.slr_wf[s,:,:,:],axis=(0,1))[-1]/(100*np.median(fe.slr_nf[s,:,:,:],axis=(0,1))[-1]))
    print(100*np.percentile(fe.slr_wf[s,:,:,:],perc[1],axis=(0,1))[-1]/(100*np.percentile(fe.slr_nf[s,:,:,:],perc[1],axis=(0,1))[-1]))
        
ax[0].legend(loc='upper left')
ax[0].set_ylabel('Sea level rise [cm]',fontsize=9)
ax[1].set_xlabel('Time [years]')

plt.savefig(f'../draftfigs/full_ensemble_{fe.year0}.png',dpi=450,facecolor='w',transparent=False)
plt.show()

#100-100*np.sum(fe.gamma<0,axis=(0,1))/(fe.gamma.shape[0]*fe.gamma.shape[1])

#gampos = np.where(fe.gamma>0,fe.gamma,0)

#np.nanmean(gampos,axis=(0,1))